### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 1


### Дисциплина: Управление ИТ-сервисами и контентом

##### Студент:  Матюхин Григорий Васильевич
##### Группа:   НПИбд-01-21

## Москва 2024
***

# Введение

Данная лабораторная работа посвящена HTTP запросам типа GET и POST, которые чаще всего
используются в открытых API различных Web-сервисов. Для лучшего усвоения теоретического
материала, необходимо вручную составить требуемые запросы и проинтерпретировать ответы,
полученные от сервера.

# Выполнение

В данной лабораторной работе я буду использовать `curl` для произведения Web-запросов,
а также другие утилиты  `bash` чтобы парсить ответы.

## GET запросы

Посылать все наши GET запросы будем на [https://httpbin.org/get](https://httpbin.org/get).
Посмоторим, что он выдает просто так.

In [1]:
curl -X GET https://httpbin.org/get -s | jq

{
  "args": {},
  "headers": {
    "Accept": "*/*",
    "Host": "httpbin.org",
    "User-Agent": "curl/8.7.1",
    "X-Amzn-Trace-Id": "Root=1-662975d5-35695aec208126bf1cd295b2"
  },
  "origin": "134.0.107.154",
  "url": "https://httpbin.org/get"
}


Теперь составим GET запрос с параметрами. Для него выведем содержимое запроса (будет отмечено символом `>`), содержимое ответа (будет отмечено символом `<`) и сам ответ сервера.

In [2]:
curl -X GET "https://httpbin.org/get?fname=Grigorii&lname=Matiukhin" -sv \
  2> >(rg "<|>") > >(jq)

> GET /get?fname=Grigorii&lname=Matiukhin HTTP/2
> Host: httpbin.org
> User-Agent: curl/8.7.1
> Accept: */*
> 
< HTTP/2 200 
< date: Wed, 24 Apr 2024 21:12:54 GMT
< content-type: application/json
< content-length: 339
< server: gunicorn/19.9.0
< access-control-allow-origin: *
< access-control-allow-credentials: true
< 
{
  "args": {
    "fname": "Grigorii",
    "lname": "Matiukhin"
  },
  "headers": {
    "Accept": "*/*",
    "Host": "httpbin.org",
    "User-Agent": "curl/8.7.1",
    "X-Amzn-Trace-Id": "Root=1-662975d6-2da944917d5cfad20eaa5dc2"
  },
  "origin": "134.0.107.154",
  "url": "https://httpbin.org/get?fname=Grigorii&lname=Matiukhin"
}


Мы видим те же самые аргументы, которые мы подали в строке запроса, в теле JSON ответа.

Опишем все заголовки, которые здесь присутствуют:
- Запрос
    - `Host` -- доменное имя сервера
    - `User-Agent` -- какая программа посылает запрос (в данном случае `curl`)
    - `Accept` -- какие типы данных можем получить в ответ (в данном случае все возможные)
    - `X-Amzn-Trace-Id` -- заголовок, котоый мы сами не посылали. Его добавляет AWS Load Balances, на стороне httpbin.org, он нужен для отслеживания запросов в логах приложений
- Ответ
    - `date` -- дата
    - `content-type` -- тип данный ответа (в данном случае JSON)
    - `content-length` -- длинна данных ответа
    - `server` -- что за программа отвечает на запросы
    - `access-control-allow-origin` -- каким сайтам сервер доверяет, чтобы делится с ними ответом на запрос. Если сайт не включен в значение данного заголовка, то содержимое ответа для него не будет видно. (в данном случае разрешены все сайты)
    - `access-control-allow-credentials` -- следует ли разрешать запросам на авторизацию из других сайтоы выполнятся (в данном случае следует)

## POST запросы

Посмотрим на сам эндпоинт сначала

In [3]:
curl -X POST https://httpbin.org/post -s -d {} | jq

{
  "args": {},
  "data": "",
  "files": {},
  "form": {
    "{}": ""
  },
  "headers": {
    "Accept": "*/*",
    "Content-Length": "2",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "httpbin.org",
    "User-Agent": "curl/8.7.1",
    "X-Amzn-Trace-Id": "Root=1-662975d6-6e1eb4082fe137b91d95c6cd"
  },
  "json": null,
  "origin": "134.0.107.154",
  "url": "https://httpbin.org/post"
}


### Параметры веб-формы

Отправим наши параметры как поля веб-формы. Для этого можно установить заголовок `Content-Type: application/x-www-form-urlencoded`, а можно и не устанавливать.
`curl` ставит его сам для POST запросов по-умолчанию.

Сравним это на следующих запросах:

In [4]:
curl -X POST https://httpbin.org/post \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "fname=Grigorii&lname=Matiukhin&stdnum=1032211403" -sv \
  2> >(rg ">") > /dev/null

> POST /post HTTP/2
> Host: httpbin.org
> User-Agent: curl/8.7.1
> Accept: */*
> Content-Type: application/x-www-form-urlencoded
> Content-Length: 48
> 


In [5]:
curl -X POST https://httpbin.org/post \
  -d "fname=Grigorii&lname=Matiukhin&stdnum=1032211403" -sv \
  2> >(rg ">") > /dev/null

> POST /post HTTP/2
> Host: httpbin.org
> User-Agent: curl/8.7.1
> Accept: */*
> Content-Length: 48
> Content-Type: application/x-www-form-urlencoded
> 


Теперь посмотрим на ответ:

In [6]:
curl -X POST https://httpbin.org/post \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "fname=Grigorii&lname=Matiukhin&stdnum=1032211403" -s | jq

{
  "args": {},
  "data": "",
  "files": {},
  "form": {
    "fname": "Grigorii",
    "lname": "Matiukhin",
    "stdnum": "1032211403"
  },
  "headers": {
    "Accept": "*/*",
    "Content-Length": "48",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "httpbin.org",
    "User-Agent": "curl/8.7.1",
    "X-Amzn-Trace-Id": "Root=1-662975d8-7ebcec33283bb6324e6c0713"
  },
  "json": null,
  "origin": "134.0.107.154",
  "url": "https://httpbin.org/post"
}


Сервер правильно понял, какие данные мы отправляли и извлек их из формы.

### Параметры как JSON

Теперь поробуем передать наши данные как JSON. Для этого нам нужно установить заголовок: `Content-Type: application/json`.

In [7]:
curl -X POST https://httpbin.org/post \
  -H "Content-Type: application/json" \
  -d '{"fname": "Grigorii", "lname": "Matiukhin", "stdnum": 1032211403}' -s | jq

{
  "args": {},
  "data": "{\"fname\": \"Grigorii\", \"lname\": \"Matiukhin\", \"stdnum\": 1032211403}",
  "files": {},
  "form": {},
  "headers": {
    "Accept": "*/*",
    "Content-Length": "65",
    "Content-Type": "application/json",
    "Host": "httpbin.org",
    "User-Agent": "curl/8.7.1",
    "X-Amzn-Trace-Id": "Root=1-662975d9-0c0d3c5e41e8937f4ef674db"
  },
  "json": {
    "fname": "Grigorii",
    "lname": "Matiukhin",
    "stdnum": 1032211403
  },
  "origin": "134.0.107.154",
  "url": "https://httpbin.org/post"
}


И в этот раз сервер понял наши намерения и извлек данные из JSON тела запроса.

# Вывод

В данной лабораторной работе мы смогли отправить базовые HTTP-запросы на веб-сервер.